# Question 2 b

## BVH Construction

In this question we are suppose to build a BVH tree.

### Algorithm

- First we neet to have centroids for each AABB of a surface.
- now we will be creating a rootnode that stores the data regarding BOX vertices and left right pointers and finall whether it is a leaf or not
- now we will be first finding the box vertices of root node.
- Now by recursion we will be counting the surfaces in the Box. if it is one we can return from there else we recurse it by sending left and right nodes
- Now we are done with the trees part. Now we need to traverse each time.
- For the traversal we need to first go on checking whether ray intersects the node. if yes go through left and right else return from there.

In [ ]:
struct B_node {
    bool is_leaf = false;
    bool extra_safety = true;
    struct B_node* right = NULL;
    struct B_node* left = NULL;

    int total_Surfaces = 0;
    int surf_index = -1;
    int mut = 0;

    Vector3f box_min = Vector3f(INFINITY, INFINITY, INFINITY);
    Vector3f box_max = Vector3f(-INFINITY, -INFINITY, -INFINITY);
};

The above the details that i am storing in my struct.

In [ ]:
void Surface ::build_AABB(B_node *root, Surface *Surfaces_array, int* index)
{
    if (this->is_done == false)
    {
        this->is_done = true;
        Vector3f AABB_min, AABB_max;
        for (auto &face : this->indices)
        {
            Vector3f p1 = this->vertices[face.x];
            Vector3f p2 = this->vertices[face.y];
            Vector3f p3 = this->vertices[face.z];

            // printf("%d | %d | %d\n", face.x, face.y, face.z);

            AABB_min.x = min_3f(p1.x, p2.x, p3.x);
            AABB_min.y = min_3f(p1.y, p2.y, p3.y);
            AABB_min.z = min_3f(p1.z, p2.z, p3.z);
            AABB_max.x = max_3f(p1.x, p2.x, p3.x);
            AABB_max.y = max_3f(p1.y, p2.y, p3.y);
            AABB_max.z = max_3f(p1.z, p2.z, p3.z);

            this->AABB_min = min_2v(this->AABB_min, AABB_min);
            this->AABB_max = max_2v(this->AABB_max, AABB_max);
            this->Centroid = Centroid_calc(AABB_min, AABB_max);
        }

        root->box_min = min_2v(this->AABB_min, root->box_min);
        root->box_max = max_2v(this->AABB_max, root->box_max);

        Surfaces_array[(*index)++] = *this;
    }
}

Here i will be Creating AABB boxes for each surface and also store the minimum
and maximum values for sake of root node.

In [ ]:
Interaction Scene::rayIntersect(Ray &ray)
{
    Interaction siFinal;

    if (donn == false)
    {
        donn = true;
        for (auto &surface : this->surfaces)
        {
            surface.build_AABB(root, Surfaces_array, &indexii);
        }

        std ::printf("Surfaces Total : %d\n", indexii);

        for (int i = 0; i < indexii; i++)
        {
            float cx = (Surfaces_array[i].AABB_max.x + Surfaces_array[i].AABB_min.x) / 2;
            float cy = (Surfaces_array[i].AABB_max.y + Surfaces_array[i].AABB_min.y) / 2;
            float cz = (Surfaces_array[i].AABB_max.z + Surfaces_array[i].AABB_min.z) / 2;

            Surfaces_array[i].Centroid = Vector3f(cx, cy, cz);
        }
        Build_BVH(root);
    }
        Traverse_and_print(root, &siFinal, ray);

    return siFinal;
}

The above code is my ray intersect funtion. it will build a BVH tree and traverse over object.

In [ ]:
B_node* Build_BVH(B_node *node)
{
    int surf_count = 0;
    int ind = 0;
    for (int i = 0; i < indexii; i++)
    {
        if (is_in(Surfaces_array[i], node) /*&& !Calculated[i]*/)
        {
            surf_count++;
            ind = i;
        }
    }

    node->total_Surfaces = surf_count;

    if (surf_count == 1)
    {
        // Calculated[ind] = true;
        node->surf_index = ind;
        node->is_leaf = true;
        return node;
    }

    if (surf_count == 0)
    {
        node->is_leaf = true;
        return node;
    }

    int axis = axis_finder(node);

    if (axis == X_axis || axis == Y_axis || axis == Z_axis)
    {
        float half;
        if (axis == X_axis)
            half = (node->box_min.x + node->box_max.x) / 2.0f;
        else if (axis == Y_axis)
            half = (node->box_min.y + node->box_max.y) / 2.0f;
        else
            half = (node->box_min.z + node->box_max.z) / 2.0f;

        B_node* left = new B_node;
        B_node* right = new B_node;

        if (axis == X_axis)
        {
            left->box_min = node->box_min;
            left->box_max = Vector3f(half, node->box_max.y, node->box_max.z);

            right->box_min = Vector3f(half, node->box_min.y, node->box_min.z);
            right->box_max = node->box_max;
        }
        else if (axis == Y_axis)
        {
            left->box_min = node->box_min;
            left->box_max = Vector3f(node->box_max.x, half, node->box_max.z);

            right->box_min = Vector3f(node->box_min.x, half, node->box_min.z);
            right->box_max = node->box_max;
        }
        else
        {
            left->box_min = node->box_min;
            left->box_max = Vector3f(node->box_max.x, node->box_max.y, half);

            right->box_min = Vector3f(node->box_min.x, node->box_min.y, half);
            right->box_max = node->box_max;
        }

        node->left = Build_BVH(left);
        node->right = Build_BVH(right);
    }

    return node;
}

The above code is used to calculate the BVH Tree. We will first check surfaces count in our node. if it is one then we can return else we will split. we will find where to split with the help of `axis_finder` 

In [ ]:
void Traverse_and_print(B_node *node, Interaction *siFinal, Ray &ray)
{
    // printf("I came these many times\n");
    if (node->is_leaf == true)
    {
        Surface temp = Surfaces_array[node->surf_index];
        Interaction si = temp.rayIntersect(ray);
        if (si.t <= ray.t)
        {
            *siFinal = si;
            ray.t = si.t;
        }
    }
    else
    {
        if (intersect_aabb(node->box_min, node->box_max, ray))
        {
            if (node->left != NULL)
            {
                Traverse_and_print(node->left, siFinal, ray);
            }
            if (node->right != NULL)
            {
                Traverse_and_print(node->right, siFinal, ray);
            }
        }
    }

    return;
}


Finally this the traversal part. we will first check each node whether our ray intersects to it or not. if it intersects we will find whether it is least distant object or not. if yes then we will store in `sifinal` else we go on traversing

### Render times

- HIGH POLY : 97156.851562 ms
- LOW POLY : 18929.783203 ms
- Donut :  65863.296875 ms
- TableTop : 14717.795898 ms
